In [1]:
# package(s) related to time, space and id
import datetime, time
import platform
import random
import os
import pathlib

# you need these dependencies (you can get these from anaconda)
# package(s) related to the simulation
import simpy

# spatial libraries 
import pyproj
import shapely.geometry
from shapely.geometry import Point
import shapely
import geopandas as gpd
import movingpandas as mpd

# package(s) for data handling
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation
import pickle

# OpenTNSIM
import opentnsim
import opentnsim.core as core
import opentnsim.graph_module as graph_module
import opentnsim.plot as plot


# dtv_backend
import dtv_backend.fis as fis
import dtv_backend.network
import dtv_backend.network.network_utilities
import dtv_backend.postprocessing
import dtv_backend.simple
import dtv_backend.simulate

# Used for mathematical functions
import math             
import numpy as np

# Used for making the graph to visualize our problem
import networkx as nx  
import requests
import io
from dtv_backend.lock import Lock, Locks

from tqdm.auto import tqdm

In [2]:
afzetting_bool = True
afzetting_node_1 = '8863326'
afzetting_node_2 = '30984545'


output_dir = pathlib.Path().resolve() / 'output'
dir_graphs = output_dir / 'graphs'
dir_graphs.mkdir(exist_ok=True, parents=True)

In [3]:
#Load lock info
url = "https://zenodo.org/records/6673604/files/FIS_locks_grouped.geojson?download=1"
resp = requests.get(url)
stream = io.BytesIO(resp.content)
locks_gdf = gpd.read_file(stream)

In [4]:
#Load graph
url = "https://zenodo.org/record/6673604/files/network_digital_twin_v0.3.pickle?download=1"
graph1 = dtv_backend.fis.load_fis_network(url).copy()

# remove edge als afzetting is True
if afzetting_bool == True:
    #remove edge
    graph1.remove_edge(afzetting_node_1, afzetting_node_2)

graph = graph_module.Graph()
graph.graph = graph1
graph.graph_info = opentnsim.utils.info(graph.graph)

In [5]:
type(graph)

opentnsim.graph_module.Graph

In [6]:
graph_dir = pathlib.Path('~').expanduser() / 'data/d-osp'
with open(graph_dir / 'network_digital_twin_v0.3.pickle', 'rb') as f:
    G = pickle.load(f)

In [7]:
# Convert nodes to GeoDataFrame
node_data = []
for node, data in graph1.nodes(data=True):
    # print(node, data)
    point = shapely.Point(data['X'], data['Y'])
    point = data['geometry']
    # point = data['Wkt']
    node_data.append({'id': node, 'geometry': point})

nodes_gdf = gpd.GeoDataFrame(node_data, crs="EPSG:4326")

# Convert edges to GeoDataFrame
edge_data = []
for u, v in graph1.edges():
    # print(graph1.nodes[u])
    point_u = graph1.nodes[u]['geometry']
    point_v = graph1.nodes[v]['geometry']
    line = shapely.LineString([point_u, point_v])
    edge_data.append({'source': u, 'target': v, 'geometry': line})

edges_gdf = gpd.GeoDataFrame(edge_data, crs="EPSG:4326")


In [8]:
nodes_gdf.to_file(graph_dir / "fis_graph.gpkg", layer="nodes", driver="GPKG")
edges_gdf.to_file(graph_dir / "fis_graph.gpkg", layer="edges", driver="GPKG")


In [9]:
len(graph1.edges)

16163

In [10]:
if afzetting_bool == True:
    # save afzetting in geopandas
    afzetting = pd.DataFrame({'A': graph1.nodes[afzetting_node_1],
    'B': graph1.nodes[afzetting_node_2]}).T
    afzetting = gpd.GeoDataFrame(afzetting, crs='EPSG:4326')
    afzetting.to_file(dir_graphs / "afzetting_locatie")

# save graph
edgelist = gpd.GeoDataFrame(nx.to_pandas_edgelist(graph.graph))
edgelist.to_file(dir_graphs / f"graph_afzetting_{afzetting_bool}")


/var/folders/1z/6xp0rndx2nqc7qx263rv42fh0000gn/T/ipykernel_69333/2130384995.py:10: UserWarning:

Column names longer than 10 characters will be truncated when saved to ESRI Shapefile.



In [11]:
cemt_classes_ordered = {
            "0": 0,
            "I": 1,
            "II": 2,
            "III": 3,
            "IV": 4,
            "IVa": 5,
            "Va": 6,
            "Vb": 7,
            "VIa": 8,
            "VIb": 9,
            "VIc": 10,
            "VIIa": 11,
}
# define synonyms:
code_synonyms = {
        "_0": "0",
        "V_A": "Va",
        "V_B": "Vb",
        "VI_A": "VIa",
        "VI_B": "VIb",
        "VI_C": "VIc",
    }

for edge in graph.graph.edges:
    # replace synonyms
    if graph.graph.edges[edge]["Code"] in code_synonyms:
        graph.graph.edges[edge]["Code"] = code_synonyms[graph.graph.edges[edge]["Code"]]


        

In [12]:
# if (pathlib.Path('~').expanduser() / 'data/ivs/ivs-2024-geocoded_filtered.pkl').is_file():
# # if os.path.isfile('data/ivs/ivs-2024-geocoded_filtered.pkl'):
#     data = pickle.load(open(pathlib.Path('~').expanduser() / 'data/ivs/ivs-2024-geocoded_filtered.pkl', 'rb'))
# else:
# # lees data in.
data = gpd.read_file(pathlib.Path('~').expanduser() / 'data/ivs/ivs-2024-geocoded.gpkg')
data.reset_index(drop = False, inplace = True, names = 'name')

# #filter data op bestaande iso datum en geometry.
data['datetime'] = pd.to_datetime(data['v05_06_begindt_evenement_iso'], format = 'ISO8601', errors = 'coerce')
data.dropna(subset = ['datetime', 'geometry'], inplace = True)
# data.to_pickle(pathlib.Path('~').expanduser() / 'data/ivs/ivs-2024-geocoded_filtered.pkl')


/var/folders/1z/6xp0rndx2nqc7qx263rv42fh0000gn/T/ipykernel_69333/915846236.py:10: FutureWarning:

In a future version of pandas, parsing datetimes with mixed time zones will raise an error unless `utc=True`. Please specify `utc=True` to opt in to the new behaviour and silence this warning. To create a `Series` with mixed offsets and `object` dtype, please use `apply` and `datetime.datetime.strptime`



In [16]:
import json
t_begin = pd.Timestamp('2023-10-10', tz='UTC')
t_end = pd.Timestamp('2023-10-12', tz='UTC')
# failed_vessels_stremming = json.load(open(output_dir / f'failed_vessels_afzetting_{True}.json', "r"))
# failed_vessels_no_stremming = json.load(open(output_dir / f'failed_vessels_afzetting_{False}.json', "r"))

condition_1 = data.datetime>=(t_begin)
condition_2 = data.datetime<(t_end)
condition_3 =  data.UNLO_bestemming != data.UNLO_herkomst
# condition_4 = ~data.name.isin(failed_vessels_stremming)
# condition_5 = ~data.name.isin(failed_vessels_no_stremming)
condition_6 = ~data.SK_CODE.isna()

idx = np.logical_and.reduce([
 condition_1,
 condition_2,
 condition_3, 
 # condition_4, 
 # condition_5,
 condition_6
])
data = data[idx]
# data.to_pickle(pathlib.Path('~').expanduser() / 'data/ivs/ivs-2024-geocoded_filtered.pkl')
len(data)

3845

In [17]:
#rename column SK
data['SK_CODE'] = data['SK_CODE'].rename({"C3l": "C3L", 
 "C2l": "C2L",
 "B04": "BO4",
 "B03": "BO3",
 "B02": "BO2",
 "B01": "BO1"})

# add cmt classe
database_rws_cemt = pd.read_json("/Users/hemert/projects/d-osp/digitaltwin-waterway/dtv_backend/data/DTV_shiptypes_database.json")
dict_rws_cemt = dict(zip(database_rws_cemt['RWS-class'], database_rws_cemt['CEMT-class']))
data['CEMT'] = data['SK_CODE'].map(dict_rws_cemt)

In [18]:
import dtv_backend.fis as fis
data['length'] = data['SK_CODE'].map(fis.rws_klasse_to_shiplength)
data['width'] = data['SK_CODE'].map(fis.rws_klasse_to_shipwidth)

In [20]:
data

,name,Jaarmaand,Jaar,Maand,Weeknr,v05_06_begindt_evenement_iso,v05_06_Begindt_evenement,UNLO_herkomst,UNLO_bestemming,v15_1_Scheepstype_RWS,...,v28_Beladingscode,v38_Vervoerd_gewicht,v30_4_Containers_TEU_S,nstr_nw,nst2007_nw,geometry,datetime,CEMT,length,width
310,310,2310,2023,10,41,2023-10-11T16:00:00+02:00,11 oktober 2023 16:00:00 uur,DEWLM,BEBRE,1,...,7,998000.0,0.0,5,10.1,"LINESTRING (6.70000 51.53333, 5.60000 51.13333)",2023-10-11 16:00:00+02:00,III,80.0,8.2
2621,2621,2310,2023,10,41,2023-10-10T05:00:00+02:00,10 oktober 2023 05:00:00 uur,BEANR,DELUH,2,...,7,770000.0,0.0,8,8.2,"LINESTRING (4.41667 51.21667, 8.43333 49.46667)",2023-10-10 05:00:00+02:00,Va,110.0,11.4
2622,2622,2310,2023,10,41,2023-10-11T17:00:00+02:00,11 oktober 2023 17:00:00 uur,NLOSS,NLLOM,1,...,1,NaN,0.0,,,"LINESTRING (5.53333 51.76667, 6.17280 51.44774)",2023-10-11 17:00:00+02:00,III,55.0,7.2
2623,2623,2310,2023,10,41,2023-10-11T17:00:00+02:00,11 oktober 2023 17:00:00 uur,NLAPN,BEANR,1,...,7,1215076.0,96.0,9,19.1,"LINESTRING (4.66667 52.13333, 4.41667 51.21667)",2023-10-11 17:00:00+02:00,Va,110.0,11.4
3471,3471,2310,2023,10,41,2023-10-10T15:00:00+02:00,10 oktober 2023 15:00:00 uur,NLAMS,FRAIS,1,...,7,1000000.0,0.0,1,4.4,"LINESTRING (4.81667 52.40000, 5.53333 43.21667)",2023-10-10 15:00:00+02:00,III,80.0,8.2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2500685,2500685,2310,2023,10,41,2023-10-10T22:00:00+02:00,10 oktober 2023 22:00:00 uur,DELUH,FRDEN,1,...,7,1001000.0,0.0,8,8.1,"LINESTRING (8.43333 49.46667, 3.40000 50.31667)",2023-10-10 22:00:00+02:00,IVa,105.0,9.5
2502139,2502139,2310,2023,10,41,2023-10-10T05:00:00+02:00,10 oktober 2023 05:00:00 uur,BEANR,DEMHG,9,...,7,1300000.0,0.0,2,2.1,"LINESTRING (4.41667 51.21667, 8.45000 49.48333)",2023-10-10 05:00:00+02:00,NaN,NaN,NaN
2502140,2502140,2310,2023,10,41,2023-10-10T16:00:00+02:00,10 oktober 2023 16:00:00 uur,NLRTM,NLZAA,0,...,7,535000.0,0.0,1,4.4,"LINESTRING (4.50000 51.91667, 4.82712 52.44435)",2023-10-10 16:00:00+02:00,NaN,NaN,NaN
2502141,2502141,2310,2023,10,41,2023-10-11T12:00:00+02:00,11 oktober 2023 12:00:00 uur,NLGET,BEBRU,1,...,7,1850000.0,0.0,6,3.5,"LINESTRING (5.96667 51.88333, 4.33333 50.83333)",2023-10-11 12:00:00+02:00,Va,110.0,11.4


In [21]:
from networkx.exception import NetworkXNoPath
import json
from tqdm import tqdm

# Make a class out of mix-ins
TransportResource = type('TransportResource', 
                         (core.Identifiable, core.ContainerDependentMovable, 
                          core.HasResource, core.Routable,
                          core.VesselProperties,
                         core.ExtraMetadata), 
                         {})
#define speed: 
def compute_v_provider(v_empty, v_full):
    return lambda x: 1

# load saved routes
path_vessel_routes = output_dir / f'vessel_routes_afzetting_{afzetting_bool}.json'
if path_vessel_routes.is_file():
    with open(path_vessel_routes, 'r') as f:
        saved_routes = json.load(f)
else:
    saved_routes = {}

# create vessels
vessels = []
failed_vessels = []
for index, row in tqdm(data.iterrows()):
    #determine path
    try:
        if str(row.name) in saved_routes.keys():
            path = saved_routes[str(row.name)]
        else:
            point_1 = fis.find_closest_node(graph.graph, Point(row.geometry.coords[0]))
            point_2 = fis.find_closest_node(graph.graph, Point(row.geometry.coords[-1]))
            path = fis.path_restricted_to_cemt_class(graph = graph.graph, 
                                            origin = point_1[0], 
                                            destination=point_2[0], 
                                            ship_cemt_classe=f"{row['CEMT']}", 
                                            ordered_cemt_classes=cemt_classes_ordered) 
        #path = nx.dijkstra_path(graph.graph, point_1[0], point_2[0], weight=compute_weight)
        #determine capacity
        capacity = max(row.v18_Laadvermogen*1000, row.v38_Vervoerd_gewicht, 1)
        data_vessel = {"env": None,
                "name": row.name,
                "type": row['v15_1_Scheepstype_RWS'],
                "B": row['width'],
                "L": row['length'],
                "route": path,
                "geometry": Point(row.geometry.coords[0]),  # lon, lat
                "capacity": capacity,
                "v": 0.5144*8, # 8 knopen
                "compute_v": compute_v_provider(v_empty=0.5144*8, v_full=0.5144*8),
                "departure_time": pd.to_datetime(row['v05_06_begindt_evenement_iso']),
                }
        vessel = TransportResource(**data_vessel)
        vessels.append(vessel)
    except NetworkXNoPath:
        failed_vessels.append(row.name)
    except ValueError:
        failed_vessels.append(row.name)
print(f"Failed vessels: {failed_vessels}")
print(f"Number of vessels: {len(vessels)}")

3845it [06:08, 10.44it/s]

Failed vessels: [310, 2621, 2622, 5776, 5784, 5785, 8083, 8084, 10357, 11729, 16264, 18566, 18567, 20932, 23134, 24051, 24052, 27838, 27839, 28724, 28728, 30187, 32541, 33401, 33402, 34898, 37267, 38164, 39564, 39565, 39566, 40423, 40424, 40425, 41865, 44238, 45096, 47440, 47442, 47443, 48993, 49870, 52156, 52157, 52159, 53648, 55953, 56807, 58254, 58255, 58257, 61404, 63727, 63731, 65162, 66061, 72286, 73171, 79287, 80149, 81596, 81597, 81598, 86281, 86282, 86283, 87129, 90873, 90874, 94077, 97793, 100131, 100133, 102388, 103251, 104721, 104722, 109269, 109270, 110122, 110128, 111552, 112411, 121877, 121878, 125650, 125652, 125653, 130228, 132570, 133437, 134926, 134927, 138207, 140445, 142853, 144341, 146583, 146584, 146585, 146586, 146588, 147445, 148935, 148936, 148937, 149774, 151248, 152093, 153533, 153534, 154393, 155853, 155854, 155856, 155857, 156676, 158153, 158154, 162672, 162673, 165045, 165877, 165878, 165879, 167322, 168156, 171884, 171885, 172747, 172749, 176576, 181988,

In [22]:
path_vessel_routes= output_dir / f'vessel_routes_afzetting_{afzetting_bool}.json'
a = {vessel.name: vessel.route for vessel in vessels}
# json.dump(a, open(path_vessel_routes, 'w'))
# json.dump(failed_vessels, open(output_dir / f'failed_vessels_afzetting_{afzetting_bool}.json', 'w'))

In [23]:
def start(env, vessel):
    while True:
        #wait untill ship will start sailing
        time_departure = time.mktime(vessel.metadata['departure_time'].timetuple())
        if env.now>time_departure:
            # print(f"Vessel {vessel.name} is starting at {time_departure} \n {vessel.metadata['departure_time']} \n current time: {env.now}")
            pass
            # print(i)
            # print(time_departure-env.now)
        yield env.timeout(max(0, time_departure-env.now))

        # start sailing
        vessel.log_entry_v0("Start sailing", env.now, "", vessel.geometry)
        if vessel.name == 100134:
            print(f"env.now: {env.now}, departure_time: {time_departure}")
        if vessel.name == 100134:
            print("Before move", env.now, vessel.name, vessel.v, vessel.route)
        yield from vessel.move()
        if vessel.name == 100134:
            print("After move", env.now, vessel.name, vessel.v)

        if vessel.name == 100134:
            print(vessel.name, vessel.geometry)
        vessel.log_entry_v0("Stop sailing", env.now, "", vessel.geometry)
        
        if vessel.geometry == nx.get_node_attributes(env.FG, "geometry")[vessel.route[-1]]:
            break

In [24]:
# Start simpy environment
simulation_start = min([vessel.metadata['departure_time'] for vessel in vessels])

env = simpy.Environment(initial_time = time.mktime(simulation_start.timetuple()))
env.epoch = time.mktime(simulation_start.timetuple())

env.FG = graph.graph

In [25]:
#create schuttijden dict
schuttijden = pd.read_csv(output_dir / 'passeertijden/passeertijden.csv', index_col=[1, 2], header=0)['mean']
schuttijden = schuttijden.rename({
    "Sluis 13": "sluis 13",
    "Sluis Amerongen": "sluis Amerongen",
    "Sluis Maasbracht": "sluis Maasbracht",
    "Sluis Schijndel": "sluis Schijndel",
    "Sluis St. Andries": "sluis St. Andries",
})
schuttijden_dict = {}
for sluis,sluiskolk  in schuttijden.index:
    if sluis not in schuttijden_dict:
        schuttijden_dict[sluis] = {}
    schuttijden_dict[sluis][sluiskolk] = schuttijden[(sluis, sluiskolk)]

#create locks
locks = Locks(env=env, schuttijden=schuttijden_dict)

In [26]:
import functools
for i, vessel in enumerate(vessels):
    # Add environment and path to the vessel
    vessel.env = env

    # add passing of a lock
    filled_pass_lock = functools.partial(locks.pass_lock, vessel=vessel)
    vessel.on_pass_edge_functions = [filled_pass_lock]

    # Add the movements of the vessel to the simulation
    env.process(start(env, vessel))

env.run(
    until=pd.Timestamp('2023-10-14 00:00:00', tz='UTC').timestamp()
)

schuttijden found for lock Julianasluis
schuttijden found for lock sluis St. Andries
schuttijden found for lock Volkeraksluizen
schuttijden found for lock sluis Maasbracht
schuttijden found for lock sluis Amerongen
schuttijden found for lock Oranjesluizen
schuttijden found for lock sluis Schijndel
schuttijden found for lock Krammersluizen
schuttijden found for lock sluis 13


KeyError: "The edge ('30984545', '8863326') is not in the graph."

In [48]:
# save the logs and trajectories of vessels
plot_dir = output_dir / 'plots_routes'
plot_dir.mkdir(exist_ok=True)

log = gpd.GeoDataFrame()
for vessel in vessels:
    if len(vessel.logbook) == 0:
        continue
    vessel_log = gpd.GeoDataFrame(vessel.logbook, geometry='Geometry', crs='EPSG:4326')
    vessel_log['trajectory_id'] = f'vessel_{vessel.name}_trip_1'
    vessel_log['object_id'] = f'vessel_{vessel.name}'
    log = pd.concat([log, vessel_log])
mpd_log = mpd.TrajectoryCollection(log, traj_id_col='trajectory_id', obj_id_col='object_id', t='Timestamp', )
# mpd_log.to_line_gdf().to_file(plot_dir / f'trajectories_afzetting_{afzetting_bool}.gpkg')

In [33]:
[(i, vessel.name) for i, vessel in enumerate(vessels)]

[(0, 2623),
 (1, 3471),
 (2, 4920),
 (3, 9469),
 (4, 11728),
 (5, 12519),
 (6, 14825),
 (7, 16265),
 (8, 16266),
 (9, 17113),
 (10, 17115),
 (11, 19448),
 (12, 20933),
 (13, 23133),
 (14, 24053),
 (15, 25507),
 (16, 25508),
 (17, 26320),
 (18, 30188),
 (19, 30189),
 (20, 31080),
 (21, 32542),
 (22, 32543),
 (23, 32544),
 (24, 33406),
 (25, 34896),
 (26, 34897),
 (27, 34899),
 (28, 37263),
 (29, 37264),
 (30, 37265),
 (31, 37266),
 (32, 38160),
 (33, 38165),
 (34, 38166),
 (35, 39563),
 (36, 40430),
 (37, 40431),
 (38, 40432),
 (39, 41866),
 (40, 44237),
 (41, 44239),
 (42, 45097),
 (43, 46535),
 (44, 47441),
 (45, 48992),
 (46, 48994),
 (47, 49876),
 (48, 51306),
 (49, 52158),
 (50, 53650),
 (51, 54493),
 (52, 58253),
 (53, 58256),
 (54, 62803),
 (55, 65161),
 (56, 65163),
 (57, 67528),
 (58, 67529),
 (59, 68437),
 (60, 72287),
 (61, 72288),
 (62, 75526),
 (63, 75532),
 (64, 76964),
 (65, 77817),
 (66, 79284),
 (67, 79285),
 (68, 79286),
 (69, 80153),
 (70, 82444),
 (71, 83917),
 (72, 

In [42]:
mpd_log.trajectories[4].df

,Message,Value,Geometry,trajectory_id,object_id
Timestamp,,,,,
2023-10-10 15:00:00.000000,Start sailing,,POINT (5.48380 52.21958),vessel_1020744_trip_1,vessel_1020744
2023-10-10 15:12:32.923475,Sailing to start,752.923475,POINT (5.48254 52.22631),vessel_1020744_trip_1,vessel_1020744


In [34]:
vessel_i = vessels[80]
vessel_i.route[-1]

'FN339'

In [37]:
nx.get_node_attributes(env.FG, "geometry")[vessel_i.route[-1]].y

52.174305954181

In [38]:
vessel_i.logbook

[{'Message': 'Start sailing',
  'Timestamp': datetime.datetime(2023, 10, 11, 16, 0),
  'Value': '',
  'Geometry': <POINT (4.5 51.917)>},
 {'Message': 'Sailing to start',
  'Timestamp': datetime.datetime(2023, 10, 11, 16, 2, 39, 864172),
  'Value': 159.86417184943824,
  'Geometry': <POINT (4.5 51.918)>}]

In [19]:
# Save the logs of the locks
lock_dfs = []
for id, lock_object in locks.locks_resources.items():
    logbook = pd.DataFrame(lock_object.logbook)
    for chamber in lock_object.chambers:
        logbook_chamber = pd.DataFrame(chamber.logbook)
        logbook = pd.concat([logbook, logbook_chamber], axis=0)
    if len(logbook) == 0:
        continue
    logbook = logbook.sort_values(by="Timestamp").reset_index(drop=True)
    lock_properties = pd.DataFrame(logbook["Value"].values.tolist())
    lock_df = pd.concat([logbook, lock_properties], axis=1)
    lock_df['lock_id'] = id
    lock_dfs.append(lock_df)

all_locks_df = pd.concat(lock_dfs, axis=0)
all_locks_df = all_locks_df.drop(columns=['Value'])
all_locks_df.to_csv(plot_dir / f'locks_afzetting_{afzetting_bool}.csv')


ValueError: No objects to concatenate

In [63]:
graph1.edges

EdgeView([('8861095', '8864054'), ('8861095', '8865831'), ('8864054', '8862449'), ('8867414', '8865307'), ('8867414', '8864869'), ('8867414', 'Berth82'), ('8865307', '8862258'), ('8865307', '8868262'), ('8864726', '8861055'), ('8864726', 'B5409_B'), ('8864726', 'L12338_B'), ('8861055', 'B55783_A'), ('8861526', '8867440'), ('8861526', '8863932'), ('8866222', '8862287'), ('8866222', 'B27017_B'), ('8862287', 'S4910_A'), ('8864009', '8867170'), ('8864009', '8863306'), ('8864009', '8866439'), ('8864009', 'B4418_A'), ('8867170', 'B47823_A'), ('8861926', '8866130'), ('8861926', '8865972'), ('8861926', '8861552'), ('8863105', '8867612'), ('8863105', '8862858'), ('8863105', '8864039'), ('8867612', '8863891'), ('8867612', '8866554'), ('8866520', '8866182'), ('8866520', '8865151'), ('8866520', '8865942'), ('8866182', '8861736'), ('8866182', '8861718'), ('8864727', '8866538'), ('8864727', '8864076'), ('8864727', '8867515'), ('8866538', 'Berth161'), ('8865600', '8866220'), ('8865600', '8862668'), (